In [11]:
%pip install reportlab

Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install cairosvg

Note: you may need to restart the kernel to use updated packages.


In [13]:
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.units import cm
from reportlab.lib import colors
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.cidfonts import UnicodeCIDFont
from reportlab.pdfbase.ttfonts import TTFont
import os
import xml.etree.ElementTree as ET
import cairosvg
import tempfile 
import shutil

In [14]:
font_path = r"C:\Windows\Fonts\Arial.ttf"
pdfmetrics.registerFont(TTFont("Arial", font_path))

# registra a fonte chinesa embutida
pdfmetrics.registerFont(UnicodeCIDFont("STSong-Light"))

# --------- 5 exemplos de caracteres (hanzi, pinyin, tradução) ----------
CHARS = [
    ("一", "yī", "um"),
    ("丨", "gǔn", "linha"),
    ("丶", "zhǔ", "pingo"),
    ("丿", "piě", "traço inclinado"),
    ("乙", "yǐ", "segundo (ciclo do céu)"),
    ("亅", "jué", "gancho"),
    ("二", "èr", "dois"),
    ("亠", "tóu", "tampa"),
    ("人", "rén", "pessoa"),
    ("儿", "ér", "filho"),
    ("入", "rù", "entrar"),
    ("八", "bā", "oito"),
    ("冂", "jiōng", "fronteira"),
    ("冖", "mì", "cobrir"),
    ("冫", "bīng", "gelo"),
    ("几", "jī", "mesa pequena"),
    ("凵", "qǔ", "recipiente"),
    ("刀", "dāo", "faca"),
    ("力", "lì", "força"),
    ("勹", "bāo", "embrulhar"),
    ("匕", "bǐ", "colher"),
    ("匚", "fāng", "caixa retangular"),
    ("匸", "xì", "esconder"),
    ("十", "shí", "dez"),
    ("卜", "bǔ", "adivinhação"),
    ("卩", "jié", "selo"),
    ("厂", "hàn", "penhasco"),
    ("厶", "sī", "privado"),
    ("又", "yòu", "de novo/mão direita"),
    ("口", "kǒu", "boca"),
    ("囗", "wéi", "cercar"),
    ("土", "tǔ", "terra"),
    ("士", "shì", "erudito"),
    ("夂", "zhǐ", "andar lentamente"),
    ("夊", "suī", "andar"),
    ("夕", "xī", "noite"),
    ("大", "dà", "grande"),
    ("女", "nǚ", "mulher"),
    ("子", "zǐ", "filho, criança"),
    ("宀", "mián", "teto"),
    ("寸", "cùn", "polegada"),
    ("小", "xiǎo", "pequeno"),
    ("尢", "wāng", "perna fraca"),
    ("尸", "shī", "cadáver"),
    ("屮", "chè", "broto"),
    ("山", "shān", "montanha"),
    ("川", "chuān", "rio"),
    ("工", "gōng", "trabalho"),
    ("己", "jǐ", "si mesmo"),
    ("巾", "jīn", "toalha"),
    ("干", "gān", "seco"),
    ("幺", "yāo", "fio pequeno"),
    ("广", "guǎng", "abrigo"),
    ("廴", "yǐn", "caminhar longo"),
    ("廾", "gǒng", "duas mãos"),
    ("弋", "yì", "dardo"),
    ("弓", "gōng", "arco"),
    ("彐", "jì", "focinho"),
    ("彡", "shān", "penas"),
    ("彳", "chì", "passos"),
    ("心", "xīn", "coração"),
    ("戈", "gē", "lança"),
    ("戶", "hù", "porta"),
    ("手", "shǒu", "mão"),
    ("支", "zhī", "ramo"),
    ("攴", "pū", "golpear"),
    ("文", "wén", "escrita"),
    ("斗", "dǒu", "colher medidora"),
    ("斤", "jīn", "machado"),
    ("方", "fāng", "quadrado"),
    ("无", "wú", "não, sem"),
    ("日", "rì", "sol, dia"),
    ("月", "yuè", "lua, mês"),
    ("木", "mù", "árvore, madeira"),
    ("欠", "qiàn", "faltar"),
    ("止", "zhǐ", "parar"),
    ("歹", "dǎi", "osso ruim"),
    ("殳", "shū", "arma de mão"),
    ("毋", "wú", "não"),
    ("比", "bǐ", "comparar"),
    ("毛", "máo", "pelo"),
    ("氏", "shì", "clã"),
    ("气", "qì", "ar"),
    ("水", "shuǐ", "água"),
    ("火", "huǒ", "fogo"),
    ("爪", "zhǎo", "garra"),
    ("父", "fù", "pai"),
    ("爻", "yáo", "linhas do I Ching"),
    ("爿", "qiáng", "meia árvore"),
    ("片", "piàn", "fatia"),
    ("牙", "yá", "dente"),
    ("牛", "niú", "boi"),
    ("犬", "quǎn", "cachorro"),
    ("玄", "xuán", "misterioso"),
    ("玉", "yù", "jade"),
    ("瓜", "guā", "melão"),
    ("瓦", "wǎ", "telha"),
    ("甘", "gān", "doce"),
    ("生", "shēng", "vida"),
    ("用", "yòng", "usar"),
    ("田", "tián", "campo"),
    ("疋", "pǐ", "perna de pano"),
    ("疒", "nè", "doença"),
    ("癶", "bō", "passo para cima"),
    ("白", "bái", "branco"),
    ("皮", "pí", "pele"),
    ("皿", "mǐn", "vasilha"),
    ("目", "mù", "olho"),
    ("矛", "máo", "lança"),
    ("矢", "shǐ", "flecha"),
    ("石", "shí", "pedra"),
    ("示", "shì", "espírito"),
    ("禸", "róu", "trilha"),
    ("禾", "hé", "grão"),
    ("穴", "xué", "caverna"),
    ("立", "lì", "ficar de pé"),
    ("竹", "zhú", "bambu"),
    ("米", "mǐ", "arroz"),
    ("糸", "mì", "fio"),
    ("缶", "fǒu", "vaso de cerâmica"),
    ("网", "wǎng", "rede"),
    ("羊", "yáng", "ovelha"),
    ("羽", "yǔ", "pena"),
    ("老", "lǎo", "velho"),
    ("而", "ér", "e"),
    ("耒", "lěi", "arado"),
    ("耳", "ěr", "orelha"),
    ("聿", "yù", "pincel"),
    ("肉", "ròu", "carne"),
    ("臣", "chén", "ministro"),
    ("自", "zì", "si mesmo"),
    ("至", "zhì", "chegar"),
    ("臼", "jiù", "pilão"),
    ("舌", "shé", "língua"),
    ("舛", "chuǎn", "contrário"),
    ("舟", "zhōu", "barco"),
    ("艮", "gèn", "duro"),
    ("色", "sè", "cor"),
    ("艸", "cǎo", "erva"),
    ("虍", "hū", "tigre"),
    ("虫", "chóng", "inseto"),
    ("血", "xuè", "sangue"),
    ("行", "xíng", "andar"),
    ("衣", "yī", "roupa"),
    ("西", "xī", "oeste"),
    ("見", "jiàn", "ver"),
    ("角", "jiǎo", "chifre"),
    ("言", "yán", "fala"),
    ("谷", "gǔ", "vale"),
    ("豆", "dòu", "feijão"),
    ("豕", "shǐ", "porco"),
    ("豸", "zhì", "animal com pernas curtas"),
    ("貝", "bèi", "concha"),
    ("赤", "chì", "vermelho"),
    ("走", "zǒu", "andar"),
    ("足", "zú", "pé"),
    ("身", "shēn", "corpo"),
    ("車", "chē", "carro"),
    ("辛", "xīn", "amargo"),
    ("辰", "chén", "tempo"),
    ("辵", "chuò", "andar"),
    ("邑", "yì", "cidade"),
    ("酉", "yǒu", "jarra de vinho"),
    ("釆", "biàn", "separar grãos"),
    ("里", "lǐ", "vilarejo"),
    ("金", "jīn", "ouro, metal"),
    ("長", "cháng", "longo"),
    ("門", "mén", "porta"),
    ("阜", "fù", "colina"),
    ("隶", "lì", "escravo"),
    ("隹", "zhuī", "ave curta"),
    ("雨", "yǔ", "chuva"),
    ("青", "qīng", "azul/verde"),
    ("非", "fēi", "não"),
    ("面", "miàn", "rosto"),
    ("革", "gé", "couro"),
    ("韋", "wéi", "couro curtido"),
    ("韭", "jiǔ", "alho-poró"),
    ("音", "yīn", "som"),
    ("頁", "yè", "página"),
    ("風", "fēng", "vento"),
    ("飛", "fēi", "voar"),
    ("食", "shí", "comida"),
    ("首", "shǒu", "cabeça"),
    ("香", "xiāng", "perfume"),
    ("馬", "mǎ", "cavalo"),
    ("骨", "gǔ", "osso"),
    ("高", "gāo", "alto"),
    ("髟", "biāo", "cabelo"),
    ("鬥", "dòu", "luta"),
    ("鬯", "chàng", "bebida sacrificial"),
    ("鬲", "lì", "caldeirão"),
    ("鬼", "guǐ", "fantasma"),
    ("魚", "yú", "peixe"),
    ("鳥", "niǎo", "pássaro"),
    ("鹵", "lǔ", "sal"),
    ("鹿", "lù", "cervo"),
    ("麥", "mài", "trigo"),
    ("麻", "má", "cânhamo"),
    ("黃", "huáng", "amarelo"),
    ("黍", "shǔ", "painço"),
    ("黑", "hēi", "preto"),
    ("黹", "zhǐ", "bordado"),
    ("黽", "mǐn", "rã"),
    ("鼎", "dǐng", "trípode"),
    ("鼓", "gǔ", "tambor"),
    ("鼠", "shǔ", "rato"),
    ("鼻", "bí", "nariz"),
    ("齊", "qí", "uniforme"),
    ("齒", "chǐ", "dente"),
    ("龍", "lóng", "dragão"),
    ("龜", "guī", "tartaruga"),
    ("龠", "yuè", "flauta")
]

# --------- Parâmetros de layout ---------
PAGE_W, PAGE_H = A4

In [15]:
# Desenha o quadrado de treino em si
# parametros: 
    # c = folha 
    # x = coordenada horizontal do canto inferior esquerdo do retângulo
    # y = coordenada vertical do canto inferior esquerdo do retângulo
    # size = largura/ altura
    
def draw_mizige(c, x, y, size, dashed=True, diagonals=True):

    # desenhando um quadrado 
    c.rect(x, y, size, size)

    c.setDash(1, 2)  # traço 1, espaço 2
    # cruz
    c.line(x, y + size/2, x + size, y + size/2)
    c.line(x + size/2, y, x + size/2, y + size)

    # diagonais
    if diagonals:
        c.line(x, y, x + size, y + size)
        c.line(x, y + size, x + size, y)
    
    if dashed:
        c.setDash()  # volta ao traço contínuo
    

In [16]:
def draw_inner_char(c, cx, cy, hanzi, size_pt, mode="ghost"):
    
    if mode == "none":
        return
    if mode == "ghost":
        c.saveState()
        c.setFont('STSong-Light', size_pt)
        c.setFillColorRGB(0.75, 0.75, 0.75)
        w = pdfmetrics.stringWidth(hanzi, 'STSong-Light', size_pt)
        c.drawString(cx - w/2, cy - size_pt*0.35, hanzi)
        c.restoreState()
    elif mode == "dashed":
        c.saveState()
        text = c.beginText()
        text.setFont('STSong-Light', size_pt)
        text.setTextRenderMode(1)    # contorno apenas
        c.setLineWidth(0.8)
        c.setDash(1, 2)
        w = pdfmetrics.stringWidth(hanzi, 'STSong-Light', size_pt)
        text.setTextOrigin(cx - w/2, cy - size_pt*0.35)
        text.textLine(hanzi)
        c.drawText(text)
        c.setDash()
        c.restoreState()

In [17]:
def get_strokes_from_svg(svg_file):
    """Extrai a lista de traços (path d=...) de um SVG do makemeahanzi"""
    tree = ET.parse(svg_file)
    root = tree.getroot()
    ns = {'svg': 'http://www.w3.org/2000/svg'}
    paths = [elem.attrib['d'] for elem in root.findall('.//svg:path', ns)]
    return paths

In [18]:
def generate_step_pngs(svg_file, out_folder, size=80):
    """Gera PNGs cumulativos (traço a traço) em uma pasta temporária"""
    strokes = get_strokes_from_svg(svg_file)
    step_files = []
    for step in range(1, len(strokes)+1):
        svg_content = f'''
        <svg xmlns="http://www.w3.org/2000/svg" viewBox="0 0 1024 1024">
          <g transform="scale(1,-1) translate(0,-900)">
            {''.join([f'<path d="{st}" stroke="black" stroke-width="60" fill="none"/>' for st in strokes[:step]])}
          </g>
        </svg>
        '''
        out_file = os.path.join(out_folder, f"step_{step}.png")
        cairosvg.svg2png(bytestring=svg_content.encode('utf-8'),
                         write_to=out_file,
                         output_width=size,
                         output_height=size)
        step_files.append(out_file)
    return step_files

In [ ]:
def write_chars(c, chars, start_x=40, start_y=None, line_gap=70, svg_folder="svgs"):
    
    if start_y is None:
        start_y = PAGE_H - 45

    y = start_y
    for hanzi, pinyin, meaning in chars:

        # --- checa se cabe na página ---
        if y - 50 < 0:        # se não há espaço suficiente
            c.showPage()       # cria nova página
            y = PAGE_H - 45    # reseta posição no topo da página

        # escreve pinyin + tradução
        c.setFont("Arial", 11)
        texto = f"{pinyin} - {meaning} "
        c.drawString(start_x, y, texto)

        # escreve o hanzi logo abaixo
        c.setFont("STSong-Light", 23)
        c.drawString(start_x, y - 40, hanzi)

        # mede a largura do hanzi
        w = pdfmetrics.stringWidth(hanzi, "STSong-Light", 23)

        # coordenadas do quadrado (lado direito do hanzi)
        s = 1.3 * cm
        x_mizige = start_x + w + 10
        y_mizige = (y - 20) - 30   # alinha verticalmente

        for k in range(11):  # 3 quadrados
            draw_mizige(c, x_mizige + k*(s + 10), y_mizige, s,
                        dashed=True, diagonals=True)
            if k <= 3:
                draw_inner_char(c,
                                x_mizige + k*(s + 10) + s/2,
                                y_mizige + s/2,
                                hanzi,
                                int(s*0.7),
                                mode="ghost")
        
        # cria pasta temporária para os steps
        tmp_dir = tempfile.mkdtemp()
        codepoint_dec = ord(hanzi)
        svg_file = os.path.join("svgs", f"{codepoint_dec}.svg")

        if os.path.exists(svg_file):
            steps = generate_step_pngs(svg_file, tmp_dir, size=int(s))

            steps = steps[:17]
            # desenha os steps acima da fileira
            step_x = x_mizige + 200
            step_y = y_mizige + s + 8
            for step_file in steps:
                c.drawImage(step_file, step_x, step_y, width=s * 0.45, height=s * 0.45,
                            preserveAspectRatio=True, mask='auto')
                step_x += s - 20

        # apaga pasta temporária
        shutil.rmtree(tmp_dir, ignore_errors=True)
        
        y -= line_gap

In [27]:
path=r"C:\Users\beatriz\Documents\chinese_study\PDFs\radicais.pdf"
c = canvas.Canvas(path, pagesize=A4)
write_chars(c, CHARS, svg_folder=r"C:\Users\beatriz\Documents\chinese_study\svgs")  # <- pasta onde estão os .svg do makemeahanzi

c.showPage()
c.save()
print(f"OK: {path}")
print("salvo")

OK: C:\Users\beatriz\Documents\chinese_study\PDFs\radicais.pdf
salvo
